In [12]:
from binance.client import Client
import pandas as pd
import pytz
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_ta as ta
import numpy as np

In [61]:
def total_signal(df, cur_candle):
    if cur_candle < 3:  # Ensure we have at least 4 candles to check
        return False
    last_4_candles = df.iloc[cur_candle-3:cur_candle+1]

    if all(last_4_candles['Close'] < last_4_candles['Open']):
        #if we see 4 red bars, we long
        return 1
    elif all(last_4_candles['Close'] >= last_4_candles['Open']):  #if we see 4 green bars, we shrot
        return -1
    else:
        return 0


def fetch_market_data_binance(symbol, interval,starting_date):
    info=Client().get_historical_klines(symbol=symbol, interval=interval, start_str = starting_date)
    df = pd.DataFrame(info)
    df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset vol', 'Num trades', 'Taker buy base', 'Taker buy quote', 'Ignore']
    df = df.astype(float)
    
    df['Date'] = pd.to_datetime(pd.to_numeric(df['Date']), unit='ms')
    df.set_index('Date', inplace=True)
    
    melbourne_tz = pytz.timezone('Australia/Melbourne')
    df.index = df.index.tz_localize('UTC').tz_convert(melbourne_tz)
    df.index = df.index.tz_localize(None)



    df['EMA'] = ta.ema(df['Close'], length = 9)
    macd = ta.macd(df['Close'], fast=12, slow=26, signal=9)
    df['MACD'] = macd.iloc[:, 0]  
    df['MACD_Signal'] = macd.iloc[:, 2]  
    display(df)
    return df



In [82]:
def plot_candlestick_with_signals(df, start_index):
    """
    Plots a candlestick chart with signal points.
    
    Parameters:
    df (DataFrame): DataFrame containing the stock data with 'Open', 'High', 'Low', 'Close', and 'pointpos' columns.
    start_index (int): The starting index for the subset of data to plot.
    num_rows (int): The number of rows of data to plot.
    
    Returns:
    None
    """
    df['pointpos'] = [
        float(row['Low']) - (float(row['High']) - float(row['Low'])) * 0.5 if 'TOTAL_SIGNAL' in row and row['TOTAL_SIGNAL'] == 1 else 
        float(row['High']) + (float(row['High']) - float(row['Low'])) * 0.5 if 'TOTAL_SIGNAL' in row and row['TOTAL_SIGNAL'] == -1 else 
        None
        for _, row in df.iterrows()
    ]
    df_subset = df[start_index:]
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1,  # Space between subplots
                        row_heights=[0.7, 0.3],
                        subplot_titles = ['Candlestick Chart', 'MACD'])  # Candlestick takes 70%, MACD takes 30%

    fig.add_trace(go.Candlestick(x=df_subset.index,
                                 open=df_subset['Open'],
                                 high=df_subset['High'],
                                 low=df_subset['Low'],
                                 close=df_subset['Close'],
                                 name='Candlesticks'),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=df_subset.index, y=df_subset['pointpos'], mode="markers",
                             marker=dict(size=10, color="MediumPurple", symbol='circle'),
                             name="Entry Points"),
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['EMA'], line=dict(color='orange', width=1), name='EMA'), row = 1, col = 1)

    # Signal Line

    fig.add_trace(go.Scatter(x=df_subset.index, y=df_subset['MACD'], 
                             line=dict(color='blue', width=1), 
                             name="MACD"),
                  row=2, col=1)

    fig.add_trace(go.Scatter(x=df_subset.index, y=df_subset['MACD_Signal'], 
                             line=dict(color='red', width=1), 
                             name="Signal Line"),
                  row=2, col=1)
    fig.update_layout(
        width=1200, 
        height=800, 
        plot_bgcolor='#2E2E2E',  # Dark Grey background (Light Black)
        paper_bgcolor='#2E2E2E',  # Keep the outer plot background consistent
        font=dict(color='white'),  # Use white text for better contrast


        xaxis1=dict(
            showgrid=True, 
            gridcolor='#3D3D3D',
            gridwidth=0.6, 
            zeroline=False,
            rangeslider=dict(visible=False), 
        ),

        xaxis2=dict(
            showgrid=False, gridcolor='white', zeroline=False,
            rangeslider=dict(visible=False),  
            type="date"
        ),
        yaxis1=dict(
            showgrid=True, 
            gridcolor='#3D3D3D',
            gridwidth=0.6, 
            zeroline=False,
            autorange=True, 

        ),
        yaxis2=dict(
            showgrid=False,  # ⬅️ Hide all grid lines
            zeroline=True,  # ⬅️ Keep zero line
            zerolinecolor="white",  # Make zero line visible
            zerolinewidth=1,  # Adjust thickness of zero line
  
        ),

        
        legend=dict(
            x=0.01,
            y=0.99,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="white"
            ),
            bgcolor="#2E2E2E",
            bordercolor="gray",
            borderwidth=2
        )
    )

    fig.show()




interval = Client.KLINE_INTERVAL_1HOUR
date = '1 February 2025'
df = fetch_market_data_binance('BTCUSDT', interval,date )
df['TOTAL_SIGNAL'] = [total_signal(df, i) if i >= 3 else 0 for i in range(len(df))]


plot_candlestick_with_signals(df, start_index=0)

,Open,High,Low,Close,Volume,Close time,Quote asset vol,Num trades,Taker buy base,Taker buy quote,Ignore,EMA,MACD,MACD_Signal
Date,,,,,,,,,,,,,,
2025-02-01 11:00:00,102429.56,102575.06,102213.07,102494.94,538.12176,1.738372e+12,5.512877e+07,97185.0,325.69353,3.336752e+07,0.0,NaN,NaN,NaN
2025-02-01 12:00:00,102494.94,102780.27,102460.33,102566.00,501.43830,1.738375e+12,5.146970e+07,91532.0,280.67846,2.881212e+07,0.0,NaN,NaN,NaN
2025-02-01 13:00:00,102566.00,102783.71,102400.00,102467.77,436.03518,1.738379e+12,4.472935e+07,78596.0,227.06364,2.329186e+07,0.0,NaN,NaN,NaN
2025-02-01 14:00:00,102466.96,102518.42,102198.52,102249.19,334.74761,1.738382e+12,3.425469e+07,68834.0,151.81542,1.553321e+07,0.0,NaN,NaN,NaN
2025-02-01 15:00:00,102249.19,102503.03,102118.56,102316.16,322.29531,1.738386e+12,3.297575e+07,70050.0,192.95634,1.974272e+07,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-18 17:00:00,95609.50,95641.79,95254.12,95382.75,1410.87424,1.739862e+12,1.346012e+08,219640.0,622.05262,5.934564e+07,0.0,95824.089891,-172.286909,-176.958501
2025-02-18 18:00:00,95382.74,95713.19,95298.06,95456.93,800.99003,1.739866e+12,7.648176e+07,136145.0,339.76662,3.244822e+07,0.0,95750.657913,-191.589404,-179.884682
2025-02-18 19:00:00,95456.92,95582.11,95050.00,95238.89,971.76662,1.739869e+12,9.259392e+07,135621.0,445.98545,4.249343e+07,0.0,95648.304331,-221.922592,-188.292264
